Подключение lama из локальных файлов, поскольку на соревновании запрещено подключение к сети и нет возможности скачать библиотеку 

In [1]:
!mkdir -p /kaggle/working/json2html
!cp -r /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/json2html-1.3.0/ /kaggle/working/

!pip install --no-index /kaggle/working/json2html-1.3.0/json2html-1.3.0/

!pip install --no-index --find-links=/kaggle/input/lightautoml/other/default/1/lightautoml_dependencies lightautoml

Processing ./json2html-1.3.0/json2html-1.3.0
  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7609 sha256=69d7a78fa3dba87dd91412ff17b5306fe35a229a9307762595418538887dd3b1
  Stored in directory: /root/.cache/pip/wheels/2b/6f/a3/c8d34015a7512e5e9e6a56aefb0b90eb2b1fa88a52a2b46564
Successfully built json2html
Looking in links: /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies
Processing /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/LightAutoML-0.3.7.3-py3-none-any.whl
Processing /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/AutoWoE-1.3.2-py3-none-any.whl
Processing /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/poetry_core-1.6.1-py3-none-any.whl
Processing /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl
Processing /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies

импорт всех необходимых зависимостей

In [16]:
from decimal import ROUND_HALF_UP, Decimal
import numpy as np
import pandas as pd
import warnings
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import jpx_tokyo_market_prediction

warnings.filterwarnings('ignore')

инициализация путей и констант

In [17]:
base_dir = "../input/jpx-tokyo-stock-exchange-prediction"
train_files_dir = f"{base_dir}/train_files"
supplemental_files_dir = f"{base_dir}/supplemental_files"
time_training = 14400
start_date_training = '2021-01-01'

/opt/conda/lib/python3.7/site-packages/lightautoml/transformers/categorical.py:447: RuntimeWarning: invalid value encountered in true_divide
  folds_prior = (f_sum.sum() - f_sum) / (f_count.sum() - f_count)
/opt/conda/lib/python3.7/site-packages/lightautoml/transformers/categorical.py:447: RuntimeWarning: invalid value encountered in true_divide
  folds_prior = (f_sum.sum() - f_sum) / (f_count.sum() - f_count)
/opt/conda/lib/python3.7/site-packages/lightautoml/transformers/categorical.py:447: RuntimeWarning: invalid value encountered in true_divide
  folds_prior = (f_sum.sum() - f_sum) / (f_count.sum() - f_count)
/opt/conda/lib/python3.7/site-packages/lightautoml/transformers/categorical.py:447: RuntimeWarning: invalid value encountered in true_divide
  folds_prior = (f_sum.sum() - f_sum) / (f_count.sum() - f_count)


конвертер для входных дат

In [4]:
def safe_date_convert(date_series):
    return pd.to_datetime(
        date_series.astype(str).str.slice(0, 10),
        format='%Y-%m-%d',
        errors='coerce'
    )

функции для предобработки данных

In [5]:
def adjust_price(price):
    price = price.copy()
    price['Date'] = safe_date_convert(price['Date'])

    def generate_adjusted_close(df):
        df = df.sort_values('Date', ascending=False)
        df['CumulativeAdjustmentFactor'] = df['AdjustmentFactor'].cumprod()
        df['AdjustedClose'] = (df['Close'] * df['CumulativeAdjustmentFactor']).map(
            lambda x: float(Decimal(str(x)).quantize(Decimal('0.1'), ROUND_HALF_UP))
        )
        return df.sort_values('Date').ffill()

    return price.groupby('SecuritiesCode', group_keys=False).apply(generate_adjusted_close)

In [6]:
def prepare_lama_data(price_df, fin_data=None):
    df = price_df.copy()

    df = df.sort_values(['SecuritiesCode', 'Date'])

    for lag in [1, 2, 3, 5]:
        df[f'return_{lag}day'] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(lag)

    windows = [5, 10, 20]
    for window in windows:
        df[f'ma_{window}'] = df.groupby('SecuritiesCode')['AdjustedClose'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
        df[f'ma_ratio_{window}'] = df['AdjustedClose'] / df[f'ma_{window}']

    df['volatility_5'] = df.groupby('SecuritiesCode')['return_1day'].transform(
        lambda x: x.rolling(5, min_periods=1).std()
    )

    if fin_data is not None:
        fin_data['Date'] = safe_date_convert(fin_data['Date'])
        df = df.merge(
            fin_data[['Date', 'SecuritiesCode', 'NetSales', 'Equity', 'Profit']],
            on=['Date', 'SecuritiesCode'],
            how='left'
        )
        for col in ['NetSales', 'Equity', 'Profit']:
            df[col] = df.groupby('SecuritiesCode')[col].ffill().fillna(0)

    df['ExpectedDividend'] = (df['ExpectedDividend'] > 0).astype(int)
    df = df.replace([np.inf, -np.inf], 0).fillna(0)

    return df

загрузчик тренировочных данных

In [7]:
def load_data():
    price_cols = ["Date", "SecuritiesCode", "Close", "AdjustmentFactor", "ExpectedDividend", "Target"]

    df_price = pd.concat([
        pd.read_csv(f"{train_files_dir}/stock_prices.csv", usecols=price_cols),
        pd.read_csv(f"{supplemental_files_dir}/stock_prices.csv", usecols=price_cols)
    ])

    df_price['Date'] = safe_date_convert(df_price['Date'])
    df_price = df_price[df_price['Date'] >= pd.to_datetime(start_date_training)]

    return df_price

инициализация и обучение модели

In [8]:
def init_automl(train_data: pd.DataFrame):
    train_data['SecuritiesCode'] = train_data['SecuritiesCode'].astype(str)

    train_data = train_data.sort_values(['SecuritiesCode', 'Date']).reset_index(drop=True)

    task = Task('reg', loss='mae')
    roles = {
        'target': 'Target',
        'drop': ['Date'],
        'group': ['SecuritiesCode'],
        'datetime': 'Date',
        'category': ['SecuritiesCode']
    }

    automl = TabularAutoML(
        task=task,
        timeout=time_training,
        cpu_limit=15,
        general_params={
            'use_algos': [['lgb', 'cb']],
            'group_role': 'SecuritiesCode'
        },
        reader_params={
            'cv': 3,
            'random_state': 42
        }
    )
    automl.fit_predict(train_data, roles=roles, verbose=1)
    return automl

загрузка данных

In [9]:
df_price = load_data()
df_price = adjust_price(df_price)

train_data = prepare_lama_data(df_price)

инициализация и обучение

In [10]:
automl = init_automl(train_data)

[01:42:32] Stdout logging level is INFO.
[01:42:32] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[01:42:32] Task: reg

[01:42:32] Start automl preset with listed constraints:
[01:42:32] - time: 100.00 seconds
[01:42:32] - CPU: 15 cores
[01:42:32] - memory: 16 GB

[01:42:32] Train data shape: (721881, 19)

[01:42:40] Layer 1 train process start. Time left 92.52 secs
[01:43:22] Selector_LightGBM fitting and predicting completed
[01:43:23] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[01:44:23] Time limit exceeded after calculating fold 0

[01:44:23] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -0.00045672257543525046
[01:44:23] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[01:44:23] Time left -10.67 secs

[01:44:23] Time limit exceeded. Last level models will be blended and unused pipelines will be pruned.

[01:44:23] Layer 1 training completed.

[01:44:23] Automl preset training completed in 110.68 seconds



создание среды для получения тестовых данных

In [11]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

генерация результатов для тестовых данных через апи соревнования

In [12]:
for (prices, _, financials, _, _, sample_prediction) in iter_test:
    prices['Date'] = safe_date_convert(prices['Date'])
    current_date = prices['Date'].iloc[0]

    price_cols = ['Date', 'SecuritiesCode', 'Close', 'AdjustmentFactor',
                  'ExpectedDividend', 'Target', 'CumulativeAdjustmentFactor',
                  'AdjustedClose']

    prices_cols = ['Date', 'RowId', 'SecuritiesCode', 'Open', 'High', 'Low',
                   'Volume', 'SupervisionFlag']

    global_df = pd.merge(
        df_price[price_cols],
        prices[prices_cols],
        on=['Date', 'SecuritiesCode'],
        how='outer',
        validate='one_to_one'
    )

    global_df = global_df.drop_duplicates(
        subset=['Date', 'SecuritiesCode'],
        keep='last'
    ).reset_index(drop=True)

    df_price = adjust_price(global_df)

    current_data = prepare_lama_data(df_price[df_price['Date'] <= current_date])
    current_data = current_data[current_data['Date'] == current_date]

    current_data = current_data.drop(columns=['Target'], errors='ignore')

    if not current_data.empty:
        current_data['SecuritiesCode'] = current_data['SecuritiesCode'].astype(str)

        current_data = current_data.sort_values(['SecuritiesCode', 'Date']).reset_index(drop=True)

        predictions = automl.predict(current_data)
        current_data['pred'] = predictions.data[:, 0]

        ranked = current_data.sort_values('pred', ascending=False)

        sample_prediction['Rank'] = ranked.index.argsort().argsort()
    else:
        sample_prediction['Rank'] = np.arange(len(sample_prediction))
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [13]:
! head submission.csv

Date,SecuritiesCode,Rank
2021-12-06,1301,125
2021-12-06,1332,632
2021-12-06,1333,648
2021-12-06,1375,701
2021-12-06,1376,1360
2021-12-06,1377,515
2021-12-06,1379,446
2021-12-06,1381,523
2021-12-06,1407,499


In [14]:
! tail submission.csv

2021-12-07,9982,559
2021-12-07,9983,146
2021-12-07,9984,1976
2021-12-07,9987,571
2021-12-07,9989,1810
2021-12-07,9990,350
2021-12-07,9991,1118
2021-12-07,9993,1825
2021-12-07,9994,193
2021-12-07,9997,300
